In [2]:
import psycopg2
import nltk
import pandas as pd
import numpy as np
import re
import spacy
import string
import pyodbc
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')
nltk.download('stopwords')
#Tilføjede kommentar

[nltk_data] Downloading package punkt to /Users/Master/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Master/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def execQuery(query):
    try:
        connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database = "postgres")
        cursor = connection.cursor()
        cursor.execute(query)
        record = cursor.fetchall()
        return record
    except (Exception, psycopg2.Error) as error :
        connection = False
        print ("Error while connecting to PostgreSQL", error)
    finally:
        if(connection):
            cursor.close()
            connection.close()
            # Det er vigtigt at tilføje 'curser.commit()' hvis man vil lave andre oparationer end at læse fra databasen
            print("Executed query and closed connection.")

In [ ]:
articlesContent = execQuery("""Select content
from article""")

In [30]:
#connection = psycopg2.connect(user = "postgres",
#                                      password = "root",
#                                      host = "localhost",
#                                      port = "5432",
#                                      database = "postgres")

# usage: specify file location, sample size and seed(used by random) 
filepath = 'small.csv'
#filepath = 'news_sample.csv' # <- overwrite for setup
s = 250                    # desired sample size(seems to have slack ie. not exact)
seed = 1                     # seed used by Pseudorandom number generator

df = pd.read_csv(filepath, index_col = [0]).sample(n=s, random_state=seed)
df["content"] = df["content"].astype(str)

In [166]:
# working with content as a series
series_content = df['content']

### preprocessing - https://www.kaggle.com/sudalairajkumar/getting-started-with-text-preprocessing ###
# try creating super regex ie. combine alle into one - might be alot faster

### replace [name] with tokens in text ###

# replace URLs  with token
regexURL= r"(?:https?:\/\/)?(?:www\.)?([^@\s]+\.[a-zA-Z]{2,4})[^\s]*"
series_content = series_content.replace(to_replace=regexURL, value='<URL>', regex=True)

# replace emails with token
regexEmail = r"[a-zA-Z_-]+@[a-zA-Z_-]+(?:\.[a-zA-Z]{2,4}){1,3}"
series_content = series_content.replace(to_replace=regexEmail, value='<EMAIL>', regex=True)

# replace dates with token
regexDate = r"(((19[7-9]\d|20\d{2})|(?:jan(?:uary)?|feb(?:ruary)?|mar(?:ch)?|apr(?:il)?|may|jun(?:e)?|jul(?:y)?|aug(?:ust)?|sep(?:tember)?|oct(?:ober)?|(nov|dec)(?:ember)?)|(([12][0-9])|(3[01])|(0?[1-9])))[\/. \-,\n]){2,3}"
series_content = series_content.replace(to_replace=regexDate, value='<DATE>', regex=True)

# repalce numbers with token
regexNum = r"[0-9][0-9,\.]+" # type of nums with [0-9] and [,.]
series_content = series_content.replace(to_replace=regexNum, value='<NUM>', regex=True)

### remove chars in text ###
# remove "\n"
regexNL = r"(\\n)"
series_content = series_content.replace(to_replace=regexNL, value='', regex=True)

# remove punctuations - remove "<" and ">"
regexPunkt = r"[!“”\"#$%&()*+,\-–.\/:;<=>?@[\\\]^_`{|}~'\’\']" # also removes: " ' " which means it's -> its - remove \' from regex if not needed
series_content = series_content.replace(to_replace=regexPunkt, value='', regex=True)
print("->", series_content.iloc[0])

# lower Casing - IMP lower casing is DONE my most of the modern day vecotirzers and tokenizers 
series_content = series_content.str.lower()

# remove stopwords - NOT DONE
stopWords = set(stopwords.words('english'))
series_content.str.split().explode().isin(stopWords) == False


series_content.head()

-> The United States has slashed its contribution to the UN Relief and Works Agency for Palestinian refugees by more than NUM million but still gave the agency NUM million Thats NUM million too much UNRWA has become the poster child for UN bureaucratic bloat mission creep and twisted morality

UNRWA was founded in NUM to help Palestinian refugees displaced by Israels war of independence Even that decision was questionable After all putting Israels creation aside in the NUMs alone wars created more than NUM million refugees There was World War II of course but the NUM partition of India also displaced NUM million persons and created millions of refugees In the wake of Israels creation Arab states expelled hundreds of thousands of their Jews This is why the UN created the United Nations High Commissioner for Refugees to be the permanent agency charged with refugee assistance and protection

Individual Palestinian Arab families suffered dislocation and tragedy but they were hardly alone T

67     the united states has slashed its contribution...
249    former us president bill clinton on monday cal...
230    the real story behind marijuana prohibition\n\...
161    audio sensors market analysis by current indus...
91     headline bitcoin  blockchain searches exceed t...
Name: content, dtype: object

In [22]:
import nltk
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer

# READ ABOUT TOKENIZERS
# https://medium.com/biaslyai/beginners-guide-to-text-preprocessing-in-python-2cbeafbf5f44
# https://www.datacamp.com/community/tutorials/text-analytics-beginners-nltk
# https://towardsdatascience.com/nlp-for-beginners-cleaning-preprocessing-text-data-ae8e306bef0f
# https://towardsdatascience.com/an-introduction-to-tweettokenizer-for-processing-tweets-9879389f8fe7

# COMAPRE TOKENIZERS
# https://miro.medium.com/max/1400/1*FLVWAVL1pkAOpN9CoVBehA.png

# series_of_str -> [ nltk.word_tokenize(string) for string in words ]

df_1 = pd.DataFrame({'content': [
                        'the sky is blue',
                        'blue not is sky the']})

# tokenization of strings - [not fast if using apply: maybe use library for faster apply or list comwprehension]
def str_tokenizer(str_words):
    # Word Tokenization
    list_words = word_tokenize(str_words)

    # Word Lemmatization
    lemmatizer = WordNetLemmatizer()
    lem_words = [lemmatizer.lemmatize(w) for w in list_words]

    # Word Stemming
    # snowball = SnowballStemmer(language = 'english')
    # stem_words = [snowball.stem(w) for w in list_words]
        
    return lem_words

# choose df-column to tokenize
series_1 = df_1['content'].apply(str_tokenizer)
print(series_1)


# list comprehension
[ nltk.word_tokenize(string) for string in df_1['content'] ]

df_1['content'] = df_1['content']

0         [the, sky, is, blue]
1    [blue, not, is, sky, the]
Name: content, dtype: object
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danie\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0        the sky is blue
1    blue not is sky the
Name: content, dtype: object

In [129]:
import numpy as np
import nltk

# df_1 = pd.DataFrame({'content': [
#                      'the sky is blue',
#                      'the sky is not blue']})

# series_1 = df_1['content'].str.split()

series_content

# used to to tokenize
    # # Tokenize and generate count vectors
    # word_vec = series_of_lists_of_tokens.str.split().apply(pd.value_counts).fillna(0)


def tf_idf(series_of_lists_of_tokens):
    """ Returns tf-idf matrix(dataframe) for a given df-column/series with list of tokens for each row. """

    # Generate count vectors
    word_vec = series_of_lists_of_tokens.apply(pd.value_counts).fillna(0) # intuitive but slow


    # Compute term frequencies
        ### Assuming from http://www.tfidf.com/ -> TF(t) = (Number of times term t appears in a document) / (Total number of terms in the document)
    tf = word_vec.divide(np.sum(word_vec, axis='columns'), axis='index')

    # Compute inverse document frequencies
        ### Slides seems wrong s.18: https://docs.google.com/presentation/d/1rXhewxbmLX7mzDcy8aucI8OLIikfrLLiz8ocN5GS06I/edit#slide=id.g77508fe5d3_0_19
        ### Assuming from http://www.tfidf.com/ -> log_e(Total number of documents / Number of documents with term t in it)
    idf = np.log(len(tf) / word_vec[word_vec > 0].count()) 

    # Compute TF-IDF vectors and return
    tfidf = np.multiply(tf, idf.to_frame().T)
    
    return tfidf

# print(tf)
# print(idf)
# stemming = PorterStemmer()
# test1 = df_1['content'].iloc[0]
# test2 = 'frightening, frightened frightens'
# series_content_tokens = df_1['content'].str.split()
# def tokenize(text):
#     list_tokens = nltk.word_tokenize(text)
#     a = [stemming.stem(word) for word in list_tokens]
#     return a

# df_tokens = series_content_tokens.explode().to_frame().reset_index().rename(columns={'index':'content_index'})
# df_tokens.groupby('content_index')[df_tokens.columns[1]].value_counts(normalize=True).unstack(level=1)


# tf_idf(series_content_tokens)
tf_idf(series_content)
# df_tokens.apply(tuple).value_counts()

df_content = pd.DataFrame(series_content)
df_content

type(df['content'])

pandas.core.series.Series

In [156]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()
x = v.fit_transform(series_content)
series_content.iloc[0]

'the united states has slashed its contribution to the un relief and works agency for palestinian refugees by more than 65 million but still gave the agency 60 million thats 60 million too much unrwa has become the poster child for un bureaucratic bloat mission creep and twisted morality\n\nunrwa was founded in 1949 to help palestinian refugees displaced by israels war of independence even that decision was questionable after all putting israels creation aside in the 1940s alone wars created more than 40 million refugees there was world war ii of course but the 1947 partition of india also displaced 14 million persons and created millions of refugees in the wake of israels creation arab states expelled hundreds of thousands of their jews this is why the un created the united nations high commissioner for refugees to be the permanent agency charged with refugee assistance and protection\n\nindividual palestinian arab families suffered dislocation and tragedy but they were hardly alone t

In [12]:
from nltk.tokenize import RegexpTokenizer
STEMMER = nltk.stem.SnowballStemmer('english')
TOKENIZER = RegexpTokenizer(r'\w+')
def textprocessing(text):
    return ''.join(STEMMER.stem(item)  for token in TOKENIZER.tokenize(text.lower()) if len(token) > 1)
a = 'frightening, frightened frightens'
textprocessing(a)

NameError: name 'item' is not defined